# Práctico 11: Navegación

La corteza entorrinal está ubicada en el lóbulo temporal medio. Es una estructura clave en el sistema de memoria y navegación espacial, ya que actúa como una interfaz entre el hipocampo y otras áreas corticales. Codifica información espacial y memoria episódica. Muchas de las neuronas de la capa II de la corteza entorrinal de las ratas son células grilla (o células de red). [Las celulas grilla son cruciales para la representación interna del espacio y la orientación de la rata](https://www.youtube.com/watch?v=8QJXDYQ4WiA). En este práctico, vamos a implementar un modelo computacional que intenta explicar mejor los mecanismos por los cuales emergen estos comportamientos.

## Configuración

Para ello, sólamente vamos a utilizar dos librerías, Numnpy y Matplotlib, por lo que procedemos a importarlas:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import requests
from matplotlib.colors import ListedColormap
from io import BytesIO

### Funciones utilitarias

In [ ]:
def descargar_trayectoria():
    url = 'https://raw.githubusercontent.com/MaestriaCienciasCognitivas/ncc/main/book/static/Practico11_HaftingTraj.npy'
    response = requests.get(url)
    response.raise_for_status()
    recordings = np.load(BytesIO(response.content))
    return recordings

### Funciones de graficado

In [ ]:
def visualizar_modelo(x, y, f, spikes, occupancy, spike_x, spike_y, spike_phases):
    cmap = plt.get_cmap('jet')
    cmap = cmap.resampled(64)
    cmap = ListedColormap([cmap(x) for x in range(32, 64)] + [cmap(x) for x in range(32)])
    
    phaseInds = np.mod(spike_phases, 2 * np.pi) * (cmap.N - 1) / (2 * np.pi)
    pointColors = cmap(np.ceil(phaseInds).astype(int))
    
    if use_real_trajectory:
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 4))
        
        ax1.plot(t, f)
        ax1.axhline(y=spike_threshold, color='r', linestyle='-')
        ax1.set_xlabel("Tiempo (s)", fontsize=9)
        ax1.set_title("Actividad (azul) y umbral (rojo)", fontsize=10)
        ax1.tick_params(axis='both', which='major', labelsize=8)
    
        ax2.imshow(np.divide(spikes, np.where(occupancy != 0, occupancy, np.nan)), origin='lower')
        ax2.set_facecolor("black")
        ax2.set_title(f"Mapa de frecuencias\nt = {tmax}s", fontsize=10)
        ax2.tick_params(axis='both', which='major', labelsize=8)
        
        ax3.scatter(spike_x, spike_y, 30 * np.ones(len(spike_x)), c=pointColors, alpha=1)
        ax3.plot(x, y, zorder=0)
        ax3.set_xlim(-1, 1)
        ax3.set_ylim(-1, 1)
        ax3.set_title("Trayectoria (azul) y\ndisparos (coloreados por fase theta)", fontsize=10)
        ax3.tick_params(axis='both', which='major', labelsize=8)
    else:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
        ax1.plot(t, f)
        ax1.set_xlabel("Tiempo (s)", fontsize=9)
        ax1.set_title("Actividad", fontsize=10)
        ax1.tick_params(axis='both', which='major', labelsize=8)
        ax2.scatter(spike_x, spike_y, 20 * np.ones(len(spike_x)), c=pointColors)
        ax2.set_ylim(-1, 1)
        ax2.set_title("Trayectoria (azul) y\ndisparos (coloreados por fase theta)", fontsize=10)
        ax2.tick_params(axis='both', which='major', labelsize=8)

## Simulación

A continuación, definimos algunas constantes globales que usaremos después:

In [ ]:
# Salto temporal, en segundos
dt = .02

# Duración total de la simulación, en segundos
tmax = 200

# Periodo de tiempo que queremos observar
t = np.arange(0, tmax, dt)

# Scaling factor relating speed to oscillator frequencies, en Hz/(m/s)
beta = 0.385

# Umbral de disparo
spike_threshold = 1.8

# Velocidad constante, en m/s, que usaremos si use_real_trajectory es False
constant_velocity = [.5, 0]

# Bines en que se separará el espacio, tanto en el eje x como en el eje y
n_spatial_bins = 60

Luego, definimos una función que integra la posición a partir de las velocidades, ya sean medidas en el laboratorio, o una constante. Luego, calcula la actividad producida por el modelo computacional a partir de la velocidad y orientación del movimiento en cada ventana temporal.

El resultado de la funcion será:

- `x`: Un vector con las posiciones en el eje x en cada paso de la integración.
- `y`: Un vector con las posiciones en el eje y en cada paso de la integración.
- `f`: Un vector con las activaciones del modelo en cada paso de la integración.
- `spikes`: Una matriz con la cantidad de activaciones que se registraron en cada bin del espacio.
- `occupancy`: Una matriz con la cantidad de veces que se registró movimiento en cada bin del espacio.
- `spike_x`: Un vector con las coordenadas en el eje x en cada activación por encima del umbral.
- `spike_y`: Un vector con las coordenadas en el eje x en cada activación por encima del umbral.
- `spike_phases`: Un vector con las fases del oscilador en cada activación por encima del umbral.

In [ ]:
# Si vamos a usar una trayectoria real, a partir de una medición tomada en el laboratorio
# o, si es False, una trayectoria calculada a partir de una velocidad constante.
# Basline maintains a fixed frequency, dorsal, en Hz
# Directional preference of each dendrite (this also sets the number of dendrites)
def integrar_modelo(use_real_trajectory = True, base_freq = 6.42, dir_preferences = np.array([0, 2*np.pi/3, 4*np.pi/3])):
    if use_real_trajectory:
        # Cargamos datos reales de navegación
        # Las primeras dos dimensiones se corresponden a los ejes x e y
        # La tercera dimension se corresponde al tiempo de cada medición
        recordings = descargar_trayectoria()
    
        # Obtenemos las posiciones, ajustadas para que la matriz tenga nuestro salto temporal dt
        time_steps = np.arange(0, recordings[2][-1], dt)
        pos = np.array([
            np.interp(time_steps, recordings[2], recordings[0]),
            np.interp(time_steps, recordings[2], recordings[1])
        ])
    
        # Pasamos de cm a m
        pos = pos / 100
        
        # Velocidad variable, en m/s
        vels = np.array([np.diff(pos[0, :]), np.diff(pos[1, :])]) / dt
    
        # Posición inicial, en m
        x0 = pos[0][0]
        y0 = pos[1][0]
    else:
        # Velocidad constante, en m/s
        vels = np.repeat([constant_velocity], len(t), axis=0).T
        
        # Posición inicial, en m
        x0 = 0
        y0 = 0
        
    # Velocidad, en m/s
    speed = np.array([])
    
    # Dirección, en radianos
    curDir = np.array([])
    
    # Posición, en metros
    x = np.array([x0])
    y = np.array([y0])
    
    # Oscillators will start at phase 0, en radianes
    dendrite_phases = np.array(np.zeros((1, len(dir_preferences))))
    base_phase = np.array([0])
    
    # Firing field plot variables
    min_x = -0.90 # en m
    max_x = 0.90 # en m
    min_y = -0.90 # en m
    max_y = 0.90 # en m
    occupancy = np.zeros((n_spatial_bins, n_spatial_bins))
    spikes = np.zeros((n_spatial_bins, n_spatial_bins))
    
    f = np.array([0])
    
    spike_times = np.array([])
    spike_x = np.array([])
    spike_y = np.array([])
    spike_phases = np.array([])
    
    for i in range(1, len(t)):
        v = vels[:, i]
        curDir = np.append(curDir, np.arctan2(v[1], v[0]))
        speed = np.append(speed, np.sqrt(np.square(v[0]) + np.square(v[1])))
        x = np.append(x, x[-1] + v[0] * dt)
        y = np.append(y, y[-1] + v[1] * dt)
        
        # Dendrite frequencies are pushed up or down from the basline frequency
        # depending on the speed and head direction, with a scaling factor
        # baseFreq*beta that sets the spacing between the spatial grid fields.
        # en Hz
        dendrite_freqs = base_freq + base_freq * beta * speed[-1] * np.cos(curDir[-1] - dir_preferences)
    
        # Advance oscillator phases
        # Radial frequency (2pi times frequency in Hz) is the time derivative of phase.
        # En rad
        dendrite_phases = np.vstack((dendrite_phases, dendrite_phases[-1] + dt * 2 * np.pi * dendrite_freqs))
        base_phase = np.append(base_phase, base_phase[-1] + dt * 2 * np.pi * base_freq)
    
        # Sum each dendritic oscillation separately with the baseline oscillation
        dendrite_plus_baseline = np.cos(dendrite_phases[-1]) + np.cos(base_phase[-1])
    
        # Final activity is the product of the oscillations.
        # Note this rule has some odd features such as positive
        # activity given an even number of negative oscillator sums and
        # the baseline is included separately in each term in the product.
        f = np.append(f, np.maximum(np.prod(dendrite_plus_baseline), 0))
        
        if f[-1] > spike_threshold:
            spike_times = np.append(spike_times, t[i])
            spike_x = np.append(spike_x, x[-1])
            spike_y = np.append(spike_y, y[-1])
            spike_phases = np.append(spike_phases, base_phase[-1])
            
        if use_real_trajectory:
            xindex = int(np.round((x[-1]-min_x)/(max_x-min_x)*n_spatial_bins))
            yindex = int(np.round((y[-1]-min_y)/(max_y-min_y)*n_spatial_bins))
            occupancy[yindex][xindex] = occupancy[yindex][xindex] + dt
            if f[-1] > spike_threshold:
                spikes[yindex][xindex] = spikes[yindex][xindex] + 1

    return x, y, f, spikes, occupancy, spike_x, spike_y, spike_phases

Ahora sí, podemos ver como se comporta el modelo según diferentes configuraciones:

In [ ]:
use_real_trajectory = True
base_freq = 6.42
dir_preferences = np.array([0, 2*np.pi/3, 4*np.pi/3])
x, y, f, spikes, occupancy, spike_x, spike_y, spike_phases = integrar_modelo(use_real_trajectory, base_freq, dir_preferences)
visualizar_modelo(x, y, f, spikes, occupancy, spike_x, spike_y, spike_phases)